# Step 1 : Web scraping 

In [44]:
df.shape

(103, 3)

# Step 2 : Finding location

**Geocoder is not working so I'll use the CSV**

In [45]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [56]:
geo.rename(columns={"Postal Code":"PostalCode"}, inplace=True)

df = df.merge(geo)

# Step 3 : Clustering

### I'll do the same thing did to New York City Data

In [57]:
toronto_longitude = -79.4163000
toronto_latitude = 43.7001100

In [73]:
# Create a map of Torondo with points
import folium

map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [74]:
CLIENT_ID = 'X2W1BIHPKDMK1EICNKQLFJK4VARYAJP1SRKU1XDKT3EIURHV' # your Foursquare ID
CLIENT_SECRET = '5HDV0SHKJKSXQCDVQT5M1KETLUH0XCL3XFDKEUWD4SV0OHSK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X2W1BIHPKDMK1EICNKQLFJK4VARYAJP1SRKU1XDKT3EIURHV
CLIENT_SECRET:5HDV0SHKJKSXQCDVQT5M1KETLUH0XCL3XFDKEUWD4SV0OHSK


In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [153]:
LIMIT = 100
venues = getNearbyVenues(names=df['PostalCode'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   radius=1000
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [154]:
venues.head()

,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
2,M1B,43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
3,M1B,43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
4,M1B,43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store


In [155]:
venues.shape

(4908, 7)

In [156]:
venues.groupby('PostalCode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,19,19,19,19,19,19
M1C,5,5,5,5,5,5
M1E,25,25,25,25,25,25
M1G,9,9,9,9,9,9
M1H,28,28,28,28,28,28
...,...,...,...,...,...,...
M9N,15,15,15,15,15,15
M9P,17,17,17,17,17,17
M9R,16,16,16,16,16,16


In [157]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 329 uniques categories.


In [158]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['PostalCode'] = venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,PostalCode,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
grouped = onehot.groupby('PostalCode').mean().reset_index()
grouped

,PostalCode,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M1B,0.0,0.00,0.052632,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,M1C,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2,M1E,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
3,M1G,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
4,M1H,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,0.0,0.035714,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,M9N,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
98,M9P,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
99,M9R,0.0,0.00,0.000000,0.0,0.0,0.0625,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
100,M9V,0.0,0.00,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [160]:
num_top_venues = 5

for hood in grouped['PostalCode']:
    print("----"+hood+"----")
    temp = grouped[grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                     venue  freq
0              Coffee Shop  0.11
1     Fast Food Restaurant  0.11
2                    Trail  0.11
3               Hobby Shop  0.05
4  Fruit & Vegetable Store  0.05


----M1C----
                venue  freq
0          Playground   0.2
1        Burger Joint   0.2
2      Breakfast Spot   0.2
3  Italian Restaurant   0.2
4                Park   0.2


----M1E----
                  venue  freq
0           Pizza Place  0.12
1           Coffee Shop  0.08
2                  Bank  0.08
3  Fast Food Restaurant  0.08
4          Burger Joint  0.04


----M1G----
                  venue  freq
0                  Park  0.22
1           Coffee Shop  0.22
2  Fast Food Restaurant  0.11
3     Indian Restaurant  0.11
4     Mobile Phone Shop  0.11


----M1H----
               venue  freq
0        Coffee Shop  0.11
1  Indian Restaurant  0.07
2        Gas Station  0.07
3             Bakery  0.07
4               Bank  0.07


----M1J----
               venue  freq
0  

                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2                Park  0.07
3  Mexican Restaurant  0.05
4               Diner  0.05


----M4S----
                       venue  freq
0                Coffee Shop  0.09
1         Italian Restaurant  0.08
2           Sushi Restaurant  0.06
3  Middle Eastern Restaurant  0.03
4                        Gym  0.03


----M4T----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.06
2       Grocery Store  0.06
3         Pizza Place  0.05
4            Pharmacy  0.05


----M4V----
                venue  freq
0         Coffee Shop  0.12
1    Sushi Restaurant  0.06
2                Park  0.05
3     Thai Restaurant  0.05
4  Italian Restaurant  0.05


----M4W----
               venue  freq
0        Coffee Shop  0.13
1      Grocery Store  0.09
2               Park  0.09
3  Korean Restaurant  0.04
4               Bank  0.04


----M4X----
         venue  freq
0         Park  0.08
1        Din

In [161]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [180]:
import numpy as np 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['PostalCode'] = grouped['PostalCode']

for ind in np.arange(grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Trail,Coffee Shop,Fast Food Restaurant,Greek Restaurant,Caribbean Restaurant,Gym,Fruit & Vegetable Store,Restaurant,Paper / Office Supplies Store,Bakery
1,M1C,Italian Restaurant,Breakfast Spot,Burger Joint,Park,Playground,Zoo,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
2,M1E,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Restaurant,Beer Store,Sandwich Place,Liquor Store,Supermarket,Juice Bar
3,M1G,Park,Coffee Shop,Mobile Phone Shop,Indian Restaurant,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Field,Filipino Restaurant,Fireworks Store
4,M1H,Coffee Shop,Gas Station,Pharmacy,Indian Restaurant,Bank,Bakery,Sporting Goods Shop,Grocery Store,Burger Joint,Fast Food Restaurant


In [181]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 2, 4, 4, 4, 2, 4, 4, 2, 2], dtype=int32)

In [182]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(venues_sorted.set_index('PostalCode'), on='PostalCode')

merged = merged.dropna()
merged["Cluster Labels"] = merged["Cluster Labels"].astype(int)

merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Trail,Coffee Shop,Fast Food Restaurant,Greek Restaurant,Caribbean Restaurant,Gym,Fruit & Vegetable Store,Restaurant,Paper / Office Supplies Store,Bakery
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Italian Restaurant,Breakfast Spot,Burger Joint,Park,Playground,Zoo,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Restaurant,Beer Store,Sandwich Place,Liquor Store,Supermarket,Juice Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Park,Coffee Shop,Mobile Phone Shop,Indian Restaurant,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Field,Filipino Restaurant,Fireworks Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Coffee Shop,Gas Station,Pharmacy,Indian Restaurant,Bank,Bakery,Sporting Goods Shop,Grocery Store,Burger Joint,Fast Food Restaurant


In [183]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Check the clusters

In [189]:
idx = 1

merged.loc[merged['Cluster Labels'] == idx, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,1,Park,Pool,Zoo,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [191]:
merged['Cluster Labels'].value_counts()

0    53
4    25
2    22
3     1
1     1
Name: Cluster Labels, dtype: int64

In [193]:
merged[merged['Cluster Labels'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M3M,North York,Downsview)Centra,43.728496,-79.495697,3,Vietnamese Restaurant,Thai Restaurant,Baseball Field,Zoo,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
